In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
bs = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [3]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim): # define architecture: simple fully connected layers
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2) # gradually expanding
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2) # gradually expanding
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim) 
    
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module): # define architecture: simple fully connected layers
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1) # the last output should be scholar (1 dimension)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [4]:
# build network
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

/home/yohei/anaconda3/envs/vae-gan01/lib/python3.8/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [5]:
G

# 784 = 28 x 28 (28 is the length of a MNIST image)

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [6]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [7]:
# loss
criterion = nn.BCELoss() # binary cross entropy loss

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [8]:
def D_train(x):
    #=======================Train the discriminator=======================#
    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D.zero_grad()
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [9]:
def G_train(x):
    #=======================Train the generator=======================#
    z = Variable(torch.randn(bs, z_dim).to(device))
    y = Variable(torch.ones(bs, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G.zero_grad()
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [10]:
n_epoch = 200
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for x, _ in train_loader:
        d_loss = D_train(x) # train discriminator
        D_losses.append(d_loss)
        g_loss = G_train(x) # train generator
        G_losses.append(g_loss)

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/200]: loss_d: 1.309, loss_g: 1.103
[2/200]: loss_d: 1.068, loss_g: 2.320
[3/200]: loss_d: 0.734, loss_g: 2.634
[4/200]: loss_d: 0.715, loss_g: 2.288
[5/200]: loss_d: 0.724, loss_g: 2.193
[6/200]: loss_d: 0.712, loss_g: 2.256
[7/200]: loss_d: 0.765, loss_g: 2.101
[8/200]: loss_d: 0.722, loss_g: 2.194
[9/200]: loss_d: 0.707, loss_g: 2.161
[10/200]: loss_d: 0.730, loss_g: 2.198
[11/200]: loss_d: 0.707, loss_g: 2.102
[12/200]: loss_d: 0.748, loss_g: 2.202
[13/200]: loss_d: 0.724, loss_g: 2.261
[14/200]: loss_d: 0.741, loss_g: 2.379
[15/200]: loss_d: 0.732, loss_g: 2.077
[16/200]: loss_d: 0.688, loss_g: 2.122
[17/200]: loss_d: 0.678, loss_g: 2.330
[18/200]: loss_d: 0.700, loss_g: 2.093
[19/200]: loss_d: 0.715, loss_g: 2.117
[20/200]: loss_d: 0.716, loss_g: 2.172
[21/200]: loss_d: 0.714, loss_g: 2.121
[22/200]: loss_d: 0.661, loss_g: 2.269
[23/200]: loss_d: 0.635, loss_g: 2.342
[24/200]: loss_d: 0.596, loss_g: 2.571
[25/200]: loss_d: 0.648, loss_g: 2.474
[26/200]: loss_d: 0.609, loss_g: 2

In [11]:
with torch.no_grad():
    test_z = Variable(torch.randn(bs, z_dim).to(device))
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + '.png')